<a href="https://colab.research.google.com/github/Gaukm01/BCS_SER_project/blob/main/mlp_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import CSVLogger

In [87]:
path='/content/drive/MyDrive/Audio'
count = 0
x_data=np.ndarray(shape=(24*60,120000))
y_data=np.ndarray(shape=(24*60,1))
for i,actor in enumerate(os.listdir(path)):
    path_new=os.path.join(path,actor)
    for j,file in enumerate(os.listdir(path_new)):
        b=file[6:8]
        y_data[count,0]=b
        y_data[count,0]=y_data[count,0]-1
        temp_path=os.path.join(path_new,file)
        # print(temp_path)
        var,_=librosa.load(temp_path)
        x_data[count,:var.shape[0]]=var[:]
        x_data[count,var.shape[0]:]=0
        count=count+1

In [88]:
sr = 22100

In [89]:
def build_mlp_model(input_shape=(180,1), num_class=8):
    
    
    model=keras.models.Sequential([keras.layers.Dropout(0.1),
                                    keras.layers.Dense(512,activation='relu',input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(1e-2)),
                                    keras.layers.Dropout(0.1),
                                    keras.layers.Dense(512,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
                                    keras.layers.Dropout(0.1),
                                    keras.layers.Dense(128,activation='relu'),
                                    keras.layers.Dense(64,activation='relu'),
                                    keras.layers.Dense(num_class,activation='softmax')
                                    ])

    return model

In [90]:
def features_extraction(file):
    result=np.array([])
    mfcc=np.mean(librosa.feature.mfcc(file,sr=sr,n_mfcc=40).T,axis=0)
    result=np.hstack((result,mfcc))
    chroma=np.mean(librosa.feature.chroma_stft(file,sr=sr).T,axis=0)
    result=np.hstack((result,chroma))
    mel=np.mean(librosa.feature.melspectrogram(file,sr=sr).T,axis=0)
    result=np.hstack((result,mel))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(file, sr=sr).T,axis=0)
    result=np.hstack((result,spectral_rolloff))
    return result

In [91]:
x=[]
for i in range(24*60):
    feature=features_extraction(x_data[i,:])
    x.append(feature)
train_X, test_X, train_y,test_y = train_test_split(np.array(x), y_data, test_size=0.2, random_state=10)

In [92]:
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

In [99]:
model=build_mlp_model()
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
csv_logger = CSVLogger('model_history.log', separator=',', append=False)

model.fit(train_X,train_y,epochs=1000,batch_size=256,validation_data=(test_X,test_y),callbacks=[csv_logger])

Epoch 1/1000
5/5 [==============================] - 1s 60ms/step - loss: 5.0896 - accuracy: 0.1892 - val_loss: 4.7845 - val_accuracy: 0.2604
Epoch 2/1000
5/5 [==============================] - 0s 26ms/step - loss: 4.5979 - accuracy: 0.3064 - val_loss: 4.3753 - val_accuracy: 0.2951
Epoch 3/1000
5/5 [==============================] - 0s 27ms/step - loss: 4.1926 - accuracy: 0.3689 - val_loss: 3.9983 - val_accuracy: 0.3750
Epoch 4/1000
5/5 [==============================] - 0s 28ms/step - loss: 3.8216 - accuracy: 0.4366 - val_loss: 3.7067 - val_accuracy: 0.3993
Epoch 5/1000
5/5 [==============================] - 0s 28ms/step - loss: 3.5039 - accuracy: 0.4523 - val_loss: 3.4058 - val_accuracy: 0.4167
Epoch 6/1000
5/5 [==============================] - 0s 25ms/step - loss: 3.2022 - accuracy: 0.4774 - val_loss: 3.1717 - val_accuracy: 0.4618
Epoch 7/1000
5/5 [==============================] - 0s 26ms/step - loss: 2.9556 - accuracy: 0.5191 - val_loss: 2.9619 - val_accuracy: 0.4861
Epoch 8/1000


In [100]:
model.evaluate(test_X,test_y)

9/9 [==============================] - 0s 3ms/step - loss: 1.4445 - accuracy: 0.7083


[1.4444642066955566, 0.7083333134651184]